# Import necessary libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix

C:\Users\Mehakpreet Singh\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Set the path to the dataset

In [2]:
train_dir = 'C:/Users/Mehakpreet Singh/Tumor/Training'
test_dir = 'C:/Users/Mehakpreet Singh/Tumor/Testing'

# Define image parameters

In [3]:
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training images

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No data augmentation for testing/validation images

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)


# Generate batches of augmented image data

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2870 images belonging to 4 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 394 images belonging to 4 classes.


# Load pre-trained MobileNetV2 model

In [8]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

C:\Users\Mehakpreet Singh\AppData\Local\Temp\ipykernel_20408\3358278115.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# Add custom layers for classification

In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Create model

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


# Compile the model

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10


C:\Users\Mehakpreet Singh\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 86s 851ms/step - accuracy: 0.6343 - loss: 0.9580 - val_accuracy: 0.6172 - val_loss: 1.3627
Epoch 2/10
 1/89 ━━━━━━━━━━━━━━━━━━━━ 46s 527ms/step - accuracy: 0.8125 - loss: 0.4868

C:\Users\Mehakpreet Singh\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8125 - loss: 0.2462 - val_accuracy: 0.7000 - val_loss: 0.7335
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 71s 771ms/step - accuracy: 0.8215 - loss: 0.4553 - val_accuracy: 0.5625 - val_loss: 1.6096
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9375 - loss: 0.1277 - val_accuracy: 0.6000 - val_loss: 0.6091
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 68s 739ms/step - accuracy: 0.8520 - loss: 0.3754 - val_accuracy: 0.6328 - val_loss: 1.4959
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8125 - loss: 0.2630 - val_accuracy: 0.6000 - val_loss: 0.5367
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 71s 773ms/step - accuracy: 0.8696 - loss: 0.3193 - val_accuracy: 0.6589 - val_loss: 1.4818
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8438 - loss: 0.2754 - val_accuracy: 0.7000 - val_loss: 0.6724
Epoch 9/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 76s 830ms/step - accuracy: 0.8751 - loss: 0.3217 - val_accuracy: 0.6562 - val_loss

# Save the model

In [15]:
model.save("brain_tumor_classification_model.keras")

In [16]:
# Load the model
loaded_model = tf.keras.models.load_model("brain_tumor_classification_model.keras")

C:\Users\Mehakpreet Singh\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 162 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
